In [1]:
import base64


def b64urlencode(s):
    return base64.urlsafe_b64encode(s).rstrip(b'=')


def b64urldecode(s):
    remainder = len(s) % 4
    if remainder == 2:
        s += b'=='
    elif remainder == 3:
        s += b'='
    elif remainder == 1:
        raise ValueError("Illegal base64 string.")
    else:
        assert remainder == 0, "Math is wrong."
    return base64.urlsafe_b64decode(s)


In [7]:
import json
from Crypto.Hash import HMAC, SHA256


def encode(payload, secret):
    header = b64urlencode(json.dumps({
        'typ': 'JWT',
        'alg': 'HS256'
    }).encode())
    payload = b64urlencode(json.dumps(payload).encode())
    hmac = HMAC.new(
        secret.encode(), 
        b'.'.join([header, payload]),
        digestmod=SHA256)
    signature = b64urlencode(hmac.digest())
    return b'.'.join([header, payload, signature])

token = encode({
    "iss": "joe",
    "exp": 1404001022
}, 'secret')


def decode(token, secret):
    protected, found, signature = token.rpartition(b'.')
    if not found:
        raise ValueError("Invalid token format.")
        
    signature = b64urldecode(signature)
    hmac = HMAC.new(secret.encode(), protected, digestmod=SHA256)
    if signature != hmac.digest():
        raise ValueError("Invalid signature.")
    
    header, payload = protected.split(b'.')
    header = json.loads(b64urldecode(header))
    payload = json.loads(b64urldecode(payload))
    

    
    
    

ExpiredSignatureError: Signature has expired.